В этом конкурсе ваша задача - разработать модель для предсказания успешности стартапа (закроется или нет)

Данные
Датасет состоит из двух файлов: тренировочный набор (около 53к записей) и тестовый набор (около 13к записей). Тренировочный набор содержит целевой признак status, указывающий на то, закрылся стартап или продолжает действовать. Временной период - '1970-01-01' по '2018-01-01'. Дата формирования выгрузки - '2018-01-01'

Описание полей данных
name - Название стартапа
category_list - Список категорий, к которым относится стартап
funding_total_usd - Общая сумма финансирования в USD
status - Статус стартапа (закрыт или действующий)
country_code - Код страны
state_code - Код штата
region - Регион
city - Город
funding_rounds - Количество раундов финансирования
founded_at - Дата основания
first_funding_at - Дата первого раунда финансирования
last_funding_at - Дата последнего раунда финансирования
closed_at - Дата закрытия стартапа (если применимо)
lifetime - Время существования стартапа в днях

In [28]:
import pandas as pd
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
!pip install fuzzywuzzy[speedup]

   ---------------------------------------- 0.0/98.8 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.8 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.8 kB 660.6 kB/s eta 0:00:01
   ----------------------------- ---------- 71.7/98.8 kB 558.5 kB/s eta 0:00:01
   ---------------------------------------- 98.8/98.8 kB 628.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.6 MB 1.7 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.6 MB 1.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.6 MB 2.7 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.6 MB 3.0 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 4.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.2/1.6 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Загрузка данных

In [3]:
train = pd.read_csv('./kaggle_startups_train_28062024.csv')
test = pd.read_csv('./kaggle_startups_test_28062024.csv')

In [4]:
train.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,828626.0,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,35456381.0,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,22250000.0,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaN


In [5]:
test.head()

,name,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,lifetime
0,Crystalsol,Clean Technology,2819200.0,NIC,17,NaN,NaN,1,2009-07-01,2009-07-01,3501
1,JBI Fish & Wings,Hospitality,NaN,USA,TN,TN - Other,Humboldt,1,2010-07-28,2010-07-28,2717
2,COINPLUS,Finance,428257.0,LUX,3,Esch-sur-alzette,Esch-sur-alzette,2,2014-05-15,2014-09-18,1295
3,Imagine Communications,Software|Video|Video Streaming,34700000.0,USA,CA,San Diego,San Diego,4,2005-01-01,2010-04-20,4748
4,DNA13,Software,4530000.0,CAN,ON,Ottawa,Ottawa,1,2007-05-08,2007-05-08,6209


## Предобработка данных

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52515 non-null  object 
 1   category_list      50051 non-null  object 
 2   funding_total_usd  42447 non-null  float64
 3   status             52516 non-null  object 
 4   country_code       47014 non-null  object 
 5   state_code         45753 non-null  object 
 6   region             46157 non-null  object 
 7   city               46157 non-null  object 
 8   funding_rounds     52516 non-null  int64  
 9   founded_at         52516 non-null  object 
 10  first_funding_at   52516 non-null  object 
 11  last_funding_at    52516 non-null  object 
 12  closed_at          4917 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 5.2+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               13125 non-null  object 
 1   category_list      12534 non-null  object 
 2   funding_total_usd  10547 non-null  float64
 3   country_code       11743 non-null  object 
 4   state_code         11430 non-null  object 
 5   region             11536 non-null  object 
 6   city               11538 non-null  object 
 7   funding_rounds     13125 non-null  int64  
 8   first_funding_at   13125 non-null  object 
 9   last_funding_at    13125 non-null  object 
 10  lifetime           13125 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.1+ MB


__Приведем поля с датами к корректоному формату__

In [8]:
for i in ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at']:
    train[i] = pd.to_datetime(train[i], errors='coerce')

In [9]:
for i in ['first_funding_at', 'last_funding_at']:
    test[i] = pd.to_datetime(test[i], errors='coerce')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               52515 non-null  object        
 1   category_list      50051 non-null  object        
 2   funding_total_usd  42447 non-null  float64       
 3   status             52516 non-null  object        
 4   country_code       47014 non-null  object        
 5   state_code         45753 non-null  object        
 6   region             46157 non-null  object        
 7   city               46157 non-null  object        
 8   funding_rounds     52516 non-null  int64         
 9   founded_at         52516 non-null  datetime64[ns]
 10  first_funding_at   52516 non-null  datetime64[ns]
 11  last_funding_at    52516 non-null  datetime64[ns]
 12  closed_at          4917 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), int64(1), object(7)
memory 

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               13125 non-null  object        
 1   category_list      12534 non-null  object        
 2   funding_total_usd  10547 non-null  float64       
 3   country_code       11743 non-null  object        
 4   state_code         11430 non-null  object        
 5   region             11536 non-null  object        
 6   city               11538 non-null  object        
 7   funding_rounds     13125 non-null  int64         
 8   first_funding_at   13125 non-null  datetime64[ns]
 9   last_funding_at    13125 non-null  datetime64[ns]
 10  lifetime           13125 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(6)
memory usage: 1.1+ MB


__Проверим датасеты на явные дубли__

In [12]:
train.duplicated().sum()

0

In [13]:
test.duplicated().sum()

0

__Явные дубли отсутствуют__

In [14]:
train[train['name'].isna()]

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
3291,NaN,Startups,25000.0,closed,USA,OR,"Portland, Oregon",Portland,1,2011-10-01,2012-03-01,2012-03-01,2013-06-03


In [15]:
train['name'].nunique()

52515

In [16]:
test['name'].nunique()

13125

__У одного стартапа в трейн не указано имя. В остальном имена уникальные__

In [17]:
train.category_list.isna().sum()

2465

In [18]:
train[train['category_list'].isna()].head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
12,Iberic Premium,NaN,107239.0,closed,NaN,NaN,NaN,NaN,1,2013-07-26,2014-06-01,2014-06-01,2015-07-10
16,Salesjob,NaN,NaN,closed,NaN,NaN,NaN,NaN,1,2013-01-01,2013-10-22,2013-10-22,2015-08-11
24,Savasti,NaN,NaN,closed,NaN,NaN,NaN,NaN,1,2015-01-01,2015-01-01,2015-01-01,2016-11-03
26,The Center from Health Promotion,NaN,10000000.0,operating,NaN,NaN,NaN,NaN,1,2002-04-21,2003-11-12,2003-11-12,NaT
64,Seamless Logistics,NaN,NaN,operating,NaN,NaN,NaN,NaN,1,2009-02-22,2011-02-01,2011-02-01,NaT


__В трейн и в тест есть ряд компаний без указанных категорий.__
- Проверил базу данных с стартапами CrunchBase, к сожалению, на примере Seamless Logistics, The Center from Health Promotion founded_at из датасетов не совпадает с указанным на сайте, а Savasti совпадает.
- API подключить не удалось, ошибка корректности ключа user. Ниже представлен код попытки
- Заполнить пропуски в категориях не представляется возможным

In [50]:
#api_key = '46fd3d384bda4c87a1da5fa7f326f16d'
#organization_name = 'savasti'
#url = f"https://api.crunchbase.com/v3.1/organizations/{organization_name}?user_key={api_key}"

# Make the API request
#response = requests.get(url)

# Handle the response
#if response.status_code == 200:
#    data = response.json()  # Parse JSON data from the response
#    print(data)             # Print or process the retrieved data
#elif response.status_code == 401:
#    print("Error 401: Unauthorized. Check if your API key is correct.")
#else:
#    print(f"Error: {response.status_code}")

In [19]:
train.category_list.nunique()

22105

In [20]:
test.category_list.nunique()

6206

In [21]:
category_list = sorted(train['category_list'].dropna().unique().tolist())
category_list

['3D',
 '3D Printing',
 '3D Printing|3D Technology',
 '3D Printing|3D Technology|Android|Artificial Intelligence|Drones|Enterprise Application|Entertainment|Entrepreneur|Games|Lasers|Mobile|Open Source|Robotics|Telecommunications',
 '3D Printing|3D Technology|Android|Games|iOS|Mobile|Mobile Games|Mobile Software Tools|Serious Games|Tablets|Video Games',
 '3D Printing|3D Technology|Audio|Mass Customization',
 '3D Printing|3D Technology|CAD|Cloud Gaming|Printing',
 '3D Printing|3D Technology|Consumer Electronics',
 '3D Printing|3D Technology|Creative Industries',
 '3D Printing|3D Technology|DIY|Printing|Software',
 '3D Printing|3D Technology|Education',
 '3D Printing|3D Technology|Graphics',
 '3D Printing|3D Technology|Manufacturing|Photography',
 '3D Printing|3D Technology|Mobile Games',
 '3D Printing|3D Technology|Startups',
 '3D Printing|Advanced Materials|Electric Vehicles|Energy|Hardware|Health Care|High Tech|Incubators|Internet of Things|Medical Devices|Robotics|Startups',
 '3D Pri

__Имеем 22 105 уникальных категорий в трейн и 6 206 в тест__

__Приведем поле category_list к списку__

In [22]:
train['category_list'] = train['category_list'].str.split('|')

In [23]:
train.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,"[Online Reservations, Restaurants]",828626.0,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaT
1,EarLens,"[Manufacturing, Medical, Medical Devices]",42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT
2,Reviva Pharmaceuticals,[Biotechnology],35456381.0,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaT
3,Sancilio and Company,[Health Care],22250000.0,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaT
4,WireTough Cylinders,[Manufacturing],NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaT


__Приведем элементы списка категорий к нижнему регистру и избавимся от возможных пробелов__

In [25]:
train['category_list'] = train['category_list'].apply(lambda x: [category.strip().lower() for category in x] if isinstance(x, list) else x)

In [30]:
train['category_list'] = train['category_list'].apply(lambda x: x if isinstance(x, list) else [])

In [26]:
train.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,"[online reservations, restaurants]",828626.0,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaT
1,EarLens,"[manufacturing, medical, medical devices]",42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT
2,Reviva Pharmaceuticals,[biotechnology],35456381.0,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaT
3,Sancilio and Company,[health care],22250000.0,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaT
4,WireTough Cylinders,[manufacturing],NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaT


In [42]:
flattened_categories = [category for sublist in train['category_list'] for category in sublist]
unique_categories = list(set(flattened_categories))

In [43]:
len(unique_categories)

854

- __Имеем 854 уникальных названий категорий__
- __Попробуем использовать библиотеку fuzzy для объеденения категорий по схожести текста__

In [44]:
def find_similar_categories(category, all_categories, threshold=85):
    matches = process.extract(category, all_categories, limit=5)  
    return [match for match, score in matches if score >= threshold]  

similar_categories = {category: find_similar_categories(category, unique_categories) for category in unique_categories}

for category, matches in similar_categories.items():
    print(f"{category}: {matches}")


digital signage: ['digital signage', 'digital rights management']
mobile search: ['mobile search', 'search', 'mobile', 'mobile emergency&health', 'mobile infrastructure']
navigation: ['navigation']
property management: ['property management', 'project management', 'intellectual asset management']
social crm: ['social crm', 'crm', 'social bookmarking', 'social network media', 'social fundraising']
field support services: ['field support services', 'services', 'bpo services', 'music services', 'local services']
3d technology: ['3d technology', 'technology', '3d', 'assisitive technology', 'deep information technology']
fleet management: ['fleet management', 'event management', 'vulnerability management', 'digital rights management', 'intellectual asset management']
digital entertainment: ['digital entertainment', 'entertainment', 'digital media']
franchises: ['franchises']
independent music labels: ['independent music labels', 'independent music', 'music', 'music education', 'music venues

__Не очень хороший результат. Например task management: ['task management', 'risk management', 'web presence management', 'social media management', 'vulnerability management'], eda tools: ['eda tools', 'developer tools', 'mobile software tools', 'customer support tools']__

__Попробуем векторизацию с помощью TF-IDF (Term Frequency-Inverse Document Frequency) и KMEANS__

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unique_categories)

num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

cluster_labels = kmeans.labels_
clustered_categories = pd.DataFrame({'category': unique_categories, 'cluster': cluster_labels})

print(clustered_categories.sort_values(by='cluster').head(60))


                            category  cluster
824              university students        0
319                     all students        0
457                      all markets        0
570             high school students        0
517          vending and concessions        1
453      bridging online and offline        1
701              sales and marketing        1
132                 wine and spirits        1
834      small and medium businesses        1
176                      oil and gas        1
137      reviews and recommendations        1
184  biotechnology and semiconductor        1
468             it and cybersecurity        1
354   embedded hardware and software        1
83              test and measurement        1
447           estimation and quoting        1
36                  video processing        2
466                  video on demand        2
379                       video chat        2
642           video game tournaments        2
115                      video gam

__Получше, но судя по первому кластеру, тоже не очень__

In [66]:
train_exploded = train.explode('category_list_split')

In [68]:
train_exploded.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at,category_list_split
0,Lunchgate,Online Reservations|Restaurants,828626.0,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaT,Online Reservations
0,Lunchgate,Online Reservations|Restaurants,828626.0,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaT,Restaurants
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT,Manufacturing
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT,Medical
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT,Medical Devices


In [69]:
train_exploded.category_list_split.value_counts()

category_list_split
Software               7017
Mobile                 4399
Biotechnology          3668
E-Commerce             3308
Curated Web            2407
                       ... 
Made in Italy             1
Private Corrections       1
Direct Advertising        1
Labor Optimization        1
Elderly                   1
Name: count, Length: 854, dtype: int64

In [10]:
train['closed_at'].describe()

count           4917
unique          3008
top       2016-12-02
freq               8
Name: closed_at, dtype: object

In [11]:
train['status'].value_counts()

status
operating    47599
closed        4917
Name: count, dtype: int64